In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install backtesting

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import plotly.graph_objects as go
import seaborn as sns
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

# Loading the data set

In [ ]:
data_xlsx = pd.read_csv('../input/bank-nifty-stock-trades/BankNiftyFutures.csv')
data_xlsx.head()

In [ ]:
data_xlsx.shape

In [ ]:
data_xlsx.info()

In [ ]:
data_xlsx

In [ ]:
data_xlsx.isnull().any()

> No null values are present, thus data is clean

In [ ]:
print(data_xlsx['Final Date format'].unique())
print("Total no. of days : " + str(len(data_xlsx['Final Date format'].unique())))

In [ ]:
data_xlsx.head()

## Time column addition so that grouping of data can be performed

In [ ]:
data_xlsx['Final Date format'] = data_xlsx['Final Date format'].astype(str)
data_xlsx['Time'] = data_xlsx['Time'].astype(str)

In [ ]:
new_format = []
date = data_xlsx['Final Date format'].values.tolist()
for i in date:
    datetimeobject = datetime.strptime(i, '%Y%m%d')
    newformat = datetimeobject.strftime('%Y-%m-%d')
    new_format.append(newformat)
    
data_xlsx['Final Date format'] = new_format

In [ ]:
data_xlsx['new_Time'] = data_xlsx['Final Date format'] + " " + data_xlsx['Time']

In [ ]:
def converttodate(l):
  dateT = []
  for strdate in l:
    date_time_obj = datetime.strptime(strdate, '%Y-%m-%d %H:%M:%S')
    dateT.append(date_time_obj)

  return dateT

In [ ]:
l = data_xlsx['new_Time'].values.tolist()
l = converttodate(l)
data_xlsx['time'] = l
data_xlsx.head()

In [ ]:
data_xlsx = data_xlsx.drop('new_Time', axis = 1)
data_xlsx.head()

# Backtesting using Python

In [ ]:
backtest = data_xlsx
backtest.head()

In [ ]:
backtest = backtest.rename(columns={'Open 5':'Open',
                                    'Close 5':'Close',
                                    'High 5':'High',
                                    'Low 5':'Low',
                                    'Volume 5':'Volume'})
backtest = backtest.set_index('time')
backtest.head()

In [ ]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()

In [ ]:
bt = Backtest(data = backtest, strategy = SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
bt.plot()

In [ ]:
df = data_xlsx.set_index('time').resample('15T', closed = 'right').sum()
df_xlsx = pd.DataFrame(df['Volume 5'])
df = data_xlsx.set_index('time').resample('15T', closed = 'right').max()
df_xlsx['high 5'] = pd.DataFrame(df['High 5'])
df = data_xlsx.set_index('time').resample('15T', closed = 'right').min()
df_xlsx['Low 5'] = pd.DataFrame(df['Low 5'])
df = data_xlsx.set_index('time').resample('15T', closed = 'right').first()
df_xlsx['Open 5'] = pd.DataFrame(df['Open 5'])
df = data_xlsx.set_index('time').resample('15T', closed = 'right').last()
df_xlsx['Close 5'] = pd.DataFrame(df['Close 5'])

df_xlsx.head()

In [ ]:
df_xlsx.isnull().any()

In [ ]:
df_xlsx.shape

In [ ]:
df_xlsx.dropna(inplace = True)

In [ ]:
df_xlsx.reset_index(inplace = True)
df_xlsx.head()

# Data converted into 15 min time frame

In [ ]:
df_xlsx

## Graph representing trades from 1 Jan 2015 - 31 Dec 2019

In [ ]:
fig = go.Figure(data = [go.Candlestick(x = df_xlsx['time'],
                open = df_xlsx['Open 5'], high = df_xlsx['high 5'],
                low = df_xlsx['Low 5'], close = df_xlsx['Close 5'])
                     ])

fig.update_layout(xaxis_rangeslider_visible = True)
fig.show()

### VWAP, TP, Traded Value, Cummulative Volume, Cummulative Traded Value calculation

In [ ]:
def calculateVwap(data):
    data['TP'] = (data['high 5']+data['Low 5']+data['Close 5'])/3.0
    data['TradedValue']  = data['TP']*data['Volume 5']
    data['CumVolume'] = data['Volume 5'].cumsum()
    data['CumTradedValue'] = data['TradedValue'].cumsum()
    data['VWAP'] = data['CumTradedValue'] /data['CumVolume']
    return data

df_xlsx = df_xlsx.groupby('time').apply(calculateVwap)

In [ ]:
df_xlsx.head()

In [ ]:
df_xlsx

# Overall Profit and Loss from 1 Jan 2015 - 31 Dec 2019

In [ ]:
print("Overall profit / loss : " + str(df_xlsx['Close 5'][30722] - df_xlsx['Open 5'][0]))
print("Overall profit / loss percentage : " + str(((df_xlsx['Close 5'][30722] - df_xlsx['Open 5'][0])/df_xlsx['Open 5'][0])*100) + " %")

### Converting data into 24 hr / per day data frame

In [ ]:
df = data_xlsx.set_index('time').resample('1440T', closed = 'right').sum()
df_day = pd.DataFrame(df['Volume 5'])
df = data_xlsx.set_index('time').resample('1440T', closed = 'right').max()
df_day['high 5'] = pd.DataFrame(df['High 5'])
df = data_xlsx.set_index('time').resample('1440T', closed = 'right').min()
df_day['Low 5'] = pd.DataFrame(df['Low 5'])
df = data_xlsx.set_index('time').resample('1440T', closed = 'right').first()
df_day['Open 5'] = pd.DataFrame(df['Open 5'])
df = data_xlsx.set_index('time').resample('1440T', closed = 'right').last()
df_day['Close 5'] = pd.DataFrame(df['Close 5'])

df_day.head(10)

In [ ]:
df_day.reset_index(inplace = True)
df_day.head(10)

In [ ]:
df_day.dropna(inplace = True)
df_day.head()

# Profit / Loss per day from 1 Jan 2015 - 31 Dec 2019

In [ ]:
df_day['Profit/Loss'] = df_day['Close 5'] - df_day['Open 5']
df_day['Profit/Loss percentage (%)'] = ((df_day['Close 5'] - df_day['Open 5'])/df_day['Open 5']) * 100
df_day.head(10)

NOTE:

> Values in Profit/Loss and Profit/Loss percentage (%) are considered to be profit sense.

> Positive value depicts a profit, similarly negative value depicts a loss.

## Graph representing profit/loss from 1 Jan 2015 - 31 Dec 2019 per day

In [ ]:
plt.figure(figsize = (40, 8))
sns.lineplot(df_day['time'], df_day['Profit/Loss'])
plt.show()

In [ ]:
df = data_xlsx.set_index('time').resample('Y', closed = 'right').sum()
df_year = pd.DataFrame(df['Volume 5'])
df = data_xlsx.set_index('time').resample('Y', closed = 'right').max()
df_year['high 5'] = pd.DataFrame(df['High 5'])
df = data_xlsx.set_index('time').resample('Y', closed = 'right').min()
df_year['Low 5'] = pd.DataFrame(df['Low 5'])
df = data_xlsx.set_index('time').resample('Y', closed = 'right').first()
df_year['Open 5'] = pd.DataFrame(df['Open 5'])
df = data_xlsx.set_index('time').resample('Y', closed = 'right').last()
df_year['Close 5'] = pd.DataFrame(df['Close 5'])

df_year

In [ ]:
df_year.reset_index(inplace = True)
df_year

# Profit / Loss per year from 1 Jan 2015 - 31 Dec 2019

In [ ]:
df_year['Profit/Loss'] = df_year['Close 5'] - df_year['Open 5']
df_year['Profit/Loss percentage (%)'] = ((df_year['Close 5'] - df_year['Open 5'])/df_year['Open 5']) * 100
df_year

NOTE:

> Values in Profit/Loss and Profit/Loss percentage (%) are considered to be profit sense.

> Positive value depicts a profit, similarly negative value depicts a loss.

## Graph representing profit/loss from 1 Jan 2015 - 31 Dec 2019 per year

In [ ]:
plt.figure(figsize = (20, 8))
sns.lineplot(df_year['time'], df_year['Profit/Loss'])
plt.show()

# Best performing year is 2017